# Wind Downscaling

## Prerequisites

* Conda environment
* Get a Copernicus API key from: https://cds.climate.copernicus.eu/api-how-to
  * create a file at \$HOME/.cdsapirc with the required UID and key

## Install the required packages

In [1]:
import os
from datetime import date, datetime
from pathlib import Path

In [2]:
if Path('./setup.py').exists():
    !pip install -e .
else:
    !pip install -U git+https://github.com/OpheliaMiralles/WindDownscaling_EPFL_UNIBE.git

  Cloning https://github.com/OpheliaMiralles/WindDownscaling_EPFL_UNIBE.git to /private/var/folders/nb/drg15dgx4mj6bqb_f4c9bnvc0000gp/T/pip-req-build-uvr3t04f
  Running command git clone -q https://github.com/OpheliaMiralles/WindDownscaling_EPFL_UNIBE.git /private/var/folders/nb/drg15dgx4mj6bqb_f4c9bnvc0000gp/T/pip-req-build-uvr3t04f


In [3]:
!conda install -y -c conda-forge gdal tensorflow xarray numpy=1.19.5 pandas pysftp cdsapi elevation rasterio dask python-dotenv

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [4]:
!pip install topo-descriptors

## Set configuration

In [5]:
DATA_ROOT = Path('/Volumes/Extreme SSD/data/')
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5_EU'
DEM_DATA_FILE = DATA_ROOT / 'dem/Europe-90m-DEM.tif'
START_DATE = date(2016,4,1)
END_DATE = date(2017,4,1)
AREA = [72,-25, 35, 65]

## Data Loading

In [7]:
from data.download_ERA5 import download_ERA5
download_ERA5(ERA5_DATA_FOLDER, START_DATE, END_DATE, AREA)
print('Done')

TypeError: download_ERA5() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
if not DEM_DATA_FILE.exists():
    dest = str(DEM_DATA_FILE)
    !eio --product SRTM3 clip -o {dest} --bounds AREA[1] AREA[0] AREA[3] AREA[2] # area of interest

## Downscaling

In [ ]:
from downscaling_utils import process_era5, process_topo, predict
import xarray as xr

## Downscaling wind fields in Switzerland with target COSMO-1

In [ ]:
HIGH_RES_TEMPLATE = xr.open_dataset('switzerland_highres_map.nc')

In [ ]:
era5 = process_era5(xr.open_dataset(ERA5_DATA_FOLDER / '20171005_era5_surface_hourly.nc'), high_res_template=HIGH_RES_TEMPLATE)
topo = process_topo(xr.open_rasterio(DEM_DATA_FILE), high_res_template=HIGH_RES_TEMPLATE)

In [ ]:
downscaled_maps = predict(era5, topo, high_res_template=HIGH_RES_TEMPLATE)

## Plots of wind fields in Switzerland

In [ ]:
from downscaling.downscaling_utils import plot_elevation, plot_wind_fields
range_long = (5.8, 10.6)
range_lat = (45.75, 47.9)
HIGH_RES_CRS = epsg(21781)  # crs for the target map, for plots

In [ ]:
raw_topo = xr.open_rasterio(DEM_DATA_FILE)
fig_topo = plot_elevation(raw_topo, range_long=range_long, range_lat=range_lat)
fig_topo.show()

In [ ]:
fig = plot_wind_fields(downscaled_maps.isel(time=0), range_long=range_long, range_lat=range_lat, high_res_crs=HIGH_RES_CRS)
fig.show()